### --- [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3) ---

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (.) and trees (#) you can see. For example:
```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
```
These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:
```
..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```
You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope right 3, down 1:

From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with O where there was an open square and X where there was a tree:
```
..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```
In this example, traversing the map using this slope would cause you to encounter 7 trees.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?


In [13]:
import csv

input_file = 'input_d3.txt'
slope = 3

with open(input_file) as f:
    reader = csv.reader(f)
    col_num = 0
    row_len = None
    trees = 0
    
    for row in reader:
        if not row_len:
            row_len = len(row[0])
        
        if row[0][col_num] == '#':
            trees += 1
        
        # print(f'{row[0][:col_num]}[{row[0][col_num]}]{row[0][col_num+1:]} : c{col_num} -> {row[0][col_num]}{trees}')
        col_num = (col_num + slope) % row_len
        
print(f'Encountered {trees} trees!')

Encountered 225 trees!


#### Part 2
Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

- Right 1, down 1.
- Right 3, down 1. (This is the slope you already checked.)
- Right 5, down 1.
- Right 7, down 1.
- Right 1, down 2.

In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [34]:
class Slope:
    """
    Simple class to represent an individual slope to check
    """
    def __init__(self, right: int, down: int):
        """
        :param right: The y-value of the slope
        :param down: The x-value of the slope
        """
        self.right = right
        self.down = down
        self.next_col = 0
        self.next_row = 0
        self.trees = 0
    
    def set_next_point(self, width):
        """
        Calculates the next point, wrapping around as appropriate
        :param width: The width of any row
        """
        # Calculate the next point 
        self.next_row += self.down
        self.next_col = (self.next_col + self.right) % width

In [38]:
def scan_slopes(slopes):
    with open(input_file) as f:
        reader = csv.reader(f)
        row_num = 0
        row_len = None

        for row in reader:
            if not row_len:
                row_len = len(row[0])
                
            # Check each slope to see if the next point is in this row
            for slope in slopes:
                if slope.next_row == row_num:
                    if row[0][slope.next_col] == '#':
                        slope.trees += 1
                    slope.set_next_point(row_len)
                    
            row_num += 1
        
        # Result is a list of num trees encountered by each slope
        result = []
        for slope in slopes:
            result.append(slope.trees)
        return result

        

In [43]:
# test against part 1
assert scan_slopes([Slope(3,1)]) == [225]

In [46]:
# Part 2 inputs
slopes = [Slope(1,1), Slope(3,1), Slope(5,1), Slope(7,1), Slope(1,2)]
tree_cts = scan_slopes(slopes)

prod = 1
for c in tree_cts:
    prod *= c
    
print(f'Tree counts: {tree_cts}, product: {prod}')

Tree counts: [60, 225, 57, 58, 25], product: 1115775000
